In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
import time
from collections import deque
import re

class WebsiteCrawler:
    def __init__(self, base_url, max_pages=100):
        self.base_url = base_url
        self.max_pages = max_pages
        self.visited_urls = set()
        self.pages_data = []
        self.domain = urlparse(base_url).netloc
        
    def is_valid_url(self, url):
        """بررسی معتبر بودن URL"""
        parsed = urlparse(url)
        return (parsed.netloc == self.domain and 
                parsed.scheme in ['http', 'https'] and
                not any(ext in url.lower() for ext in ['.pdf', '.jpg', '.png', '.gif', '.zip', '.mp4']))
    
    def extract_page_content(self, url):
        """استخراج محتوای یک صفحه"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # حذف تگ‌های غیرضروری
            for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
                tag.decompose()
            
            # استخراج اطلاعات
            title = soup.find('title').get_text() if soup.find('title') else ''
            
            # استخراج متا توضیحات
            meta_desc = ''
            meta_tag = soup.find('meta', attrs={'name': 'description'})
            if meta_tag and meta_tag.get('content'):
                meta_desc = meta_tag['content']
            
            # استخراج تمام متن صفحه
            text_content = soup.get_text(separator=' ', strip=True)
            text_content = re.sub(r'\s+', ' ', text_content)
            
            # استخراج هدینگ‌ها
            headings = []
            for i in range(1, 7):
                for heading in soup.find_all(f'h{i}'):
                    headings.append(heading.get_text(strip=True))
            
            # استخراج لینک‌ها
            links = []
            for link in soup.find_all('a', href=True):
                absolute_url = urljoin(url, link['href'])
                if self.is_valid_url(absolute_url):
                    links.append(absolute_url)
            
            return {
                'url': url,
                'title': title,
                'meta_description': meta_desc,
                'headings': headings,
                'content': text_content[:5000],  # محدود کردن محتوا
                'links': list(set(links))
            }
            
        except Exception as e:
            print(f"خطا در استخراج {url}: {str(e)}")
            return None
    
    def crawl(self):
        """خزیدن روی وبسایت"""
        queue = deque([self.base_url])
        self.visited_urls.add(self.base_url)
        
        print(f"شروع خزیدن از: {self.base_url}")
        
        while queue and len(self.pages_data) < self.max_pages:
            current_url = queue.popleft()
            print(f"در حال پردازش ({len(self.pages_data) + 1}/{self.max_pages}): {current_url}")
            
            page_data = self.extract_page_content(current_url)
            
            if page_data:
                self.pages_data.append(page_data)
                
                # اضافه کردن لینک‌های جدید به صف
                for link in page_data['links']:
                    if link not in self.visited_urls and len(self.visited_urls) < self.max_pages:
                        self.visited_urls.add(link)
                        queue.append(link)
            
            time.sleep(1)  # تاخیر برای جلوگیری از فشار به سرور
        
        print(f"\nخزیدن تمام شد. تعداد صفحات: {len(self.pages_data)}")
        return self.pages_data
    
    def save_to_file(self, filename='website_data.json'):
        """ذخیره JSON در فولدر اختصاصی سایت"""
        filepath = os.path.join(self.output_folder, filename)
        data = {
            'base_url': self.base_url,
            'total_pages': len(self.pages_data),
            'total_urls_discovered': len(self.visited_urls),
            'crawled_at': time.strftime('%Y-%m-%d %H:%M:%S'),
            'pages': self.pages_data
        }
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"ذخیره شد:\n {filepath}")
    
    def create_sitemap(self, filename='sitemap.txt'):
        """ایجاد نقشه سایت"""
        with open(filename, 'w', encoding='utf-8') as f:
            for page in self.pages_data:
                f.write(f"{page['url']}\n")
        print(f"نقشه سایت در فایل {filename} ذخیره شد")


# نحوه استفاده
if __name__ == "__main__":
    # آدرس سایت را وارد کنید
    website_url = "https://console.melipayamak.com/"
    
    # ایجاد کرالر
    crawler = WebsiteCrawler(website_url, max_pages=100)
    
    # شروع خزیدن
    crawler.crawl()
    
    # ذخیره اطلاعات
    crawler.save_to_file('website_data.json')
    crawler.create_sitemap('sitemap.txt')


کرالر پیشرفته وبسایت با Selenium
ذخیره در فولدر اختصاصی + جلوگیری از تکرار

فولدر جدید ساخته شد:
   advanced_website_crawler/console.melipayamak.com
🚀 شروع خزیدن از: https://console.melipayamak.com/
📊 حداکثر صفحات: 50


📄 [1/50] https://console.melipayamak.com/
   → بارگذاری صفحه...
   ✓ عنوان: کنسول ملی پیامک...
   ✓ محتوا: 222 کاراکتر
   ✓ لینک‌ها: 2 عدد
   ✓ 2 لینک جدید به صف اضافه شد

📄 [2/50] https://console.melipayamak.com/auth/register
   → بارگذاری صفحه...
   ✓ عنوان: کنسول ملی پیامک...
   ✓ محتوا: 244 کاراکتر
   ✓ لینک‌ها: 1 عدد
   ✓ 1 لینک جدید به صف اضافه شد

📄 [3/50] https://console.melipayamak.com/auth/forgotPassword
   → بارگذاری صفحه...
   ✓ عنوان: کنسول ملی پیامک...
   ✓ محتوا: 277 کاراکتر
   ✓ لینک‌ها: 1 عدد
   ✓ 0 لینک جدید به صف اضافه شد

📄 [4/50] https://console.melipayamak.com/auth/login
   → بارگذاری صفحه...
   ✗ خطا در استخراج https://console.melipayamak.com/auth/login: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome=141.0.7390.123)
Sta

In [ ]:
import json
import requests
from typing import List, Dict

class WebsiteQASystem:
    def __init__(self, data_file: str, api_key: str):
        """
        سیستم پرسش و پاسخ بر اساس محتوای وبسایت
        
        Args:
            data_file: مسیر فایل JSON حاوی اطلاعات وبسایت
            api_key: کلید API از openrouter.ai
        """
        self.api_key = api_key
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.model = "openai/gpt-3.5-turbo"
        
        # بارگذاری اطلاعات وبسایت
        with open(data_file, 'r', encoding='utf-8') as f:
            self.website_data = json.load(f)
        
        print(f"✓ {self.website_data['total_pages']} صفحه بارگذاری شد")
    
    def create_context(self, query: str, max_pages: int = 5) -> str:
        """ایجاد کانتکست مرتبط با سوال کاربر"""
        # جستجوی صفحات مرتبط
        relevant_pages = []
        
        query_lower = query.lower()
        for page in self.website_data['pages']:
            # امتیازدهی به صفحات
            score = 0
            content_lower = (page['title'] + ' ' + 
                           page['meta_description'] + ' ' + 
                           page['content']).lower()
            
            # جستجوی کلمات کلیدی
            for word in query_lower.split():
                if len(word) > 2:
                    score += content_lower.count(word)
            
            if score > 0:
                relevant_pages.append((score, page))
        
        # مرتب‌سازی و انتخاب بهترین صفحات
        relevant_pages.sort(reverse=True, key=lambda x: x[0])
        top_pages = [page for _, page in relevant_pages[:max_pages]]
        
        # ساخت کانتکست
        context = f"اطلاعات وبسایت {self.website_data['base_url']}:\n\n"
        
        for i, page in enumerate(top_pages, 1):
            context += f"=== صفحه {i}: {page['title']} ===\n"
            context += f"URL: {page['url']}\n"
            if page['meta_description']:
                context += f"توضیحات: {page['meta_description']}\n"
            context += f"محتوا: {page['content'][:1000]}...\n\n"
        
        return context
    
    def ask(self, question: str) -> Dict:
        """پرسیدن سوال از سیستم"""
        # ایجاد کانتکست
        context = self.create_context(question)
        
        # ساخت پیام برای مدل
        messages = [
            {
                "role": "system",
                "content": f"""تو یک دستیار هوشمند هستی که به سوالات کاربران درباره یک وبسایت پاسخ می‌دهی.
اطلاعات زیر از وبسایت استخراج شده است. فقط بر اساس این اطلاعات پاسخ بده.
اگر جوابی در اطلاعات نبود، به کاربر بگو که این اطلاعات در دسترس نیست.
پاسخ‌هایت باید دقیق، مفید و به زبان فارسی باشد.

{context}"""
            },
            {
                "role": "user",
                "content": question
            }
        ]
        
        # ارسال درخواست به API
        try:
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            
            payload = {
                "model": self.model,
                "messages": messages,
                "temperature": 0.7,
                "max_tokens": 1000
            }
            
            response = requests.post(
                self.api_url,
                headers=headers,
                json=payload,
                timeout=30
            )
            response.raise_for_status()
            
            result = response.json()
            answer = result['choices'][0]['message']['content']
            
            return {
                "success": True,
                "answer": answer,
                "model": self.model
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }
    
    def interactive_mode(self):
        """حالت تعاملی برای پرسش و پاسخ"""
        print("\n" + "="*60)
        print("سیستم پرسش و پاسخ وبسایت")
        print("="*60)
        print(f"وبسایت: {self.website_data['base_url']}")
        print(f"تعداد صفحات: {self.website_data['total_pages']}")
        print("\nبرای خروج 'exit' یا 'خروج' را تایپ کنید\n")
        
        while True:
            try:
                question = input("\n❓ سوال شما: ").strip()
                
                if question.lower() in ['exit', 'خروج', 'quit']:
                    print("خداحافظ!")
                    break
                
                if not question:
                    continue
                
                print("\n⏳ در حال پردازش...")
                result = self.ask(question)
                
                if result['success']:
                    print(f"\n✅ پاسخ:\n{result['answer']}")
                else:
                    print(f"\n❌ خطا: {result['error']}")
                    
            except KeyboardInterrupt:
                print("\n\nخداحافظ!")
                break
            except Exception as e:
                print(f"\n❌ خطا: {str(e)}")


# نحوه استفاده
if __name__ == "__main__":
    # کلید API خود را از openrouter.ai دریافت کنید
    API_KEY = "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"
    
    # ایجاد سیستم پرسش و پاسخ
    qa_system = WebsiteQASystem(
        data_file='website_data.json',
        api_key=API_KEY
    )
    
    # حالت تعاملی
    qa_system.interactive_mode()
    
    # یا استفاده مستقیم:
    # result = qa_system.ask("قیمت گوشی‌های سامسونگ چقدر است؟")
    # print(result['answer'])

In [ ]:
# pip install openai langchain faiss-cpu tiktoken
import json
from openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ────── تنظیمات تو ──────
OPENROUTER_API_KEY = "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"          # ← اینجا کلیدت رو بذار
SITE_DATA_JSON     = "site_data.json"        # فایل JSON صفحات
FAISS_INDEX_FOLDER = "site_index"            # فولدر ایندکس
# ───────────────────────

# 1. ساخت کلاینت OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

# 2. لود و آماده‌سازی داده‌های سایت
def load_site_data():
    with open(SITE_DATA_JSON, "r", encoding="utf-8") as f:
        pages = json.load(f)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = []
    for page in pages:
        chunks = splitter.split_text(page['content'])
        for i, chunk in enumerate(chunks):
            docs.append(Document(
                page_content=chunk,
                metadata={"url": page['url'], "title": page.get('title', 'بدون عنوان')}
            ))
    return docs

# 3. ساخت یا لود Vector Store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if not FAISS.index_exists(FAISS_INDEX_FOLDER):
    print("در حال ساخت ایندکس جدید...")
    docs = load_site_data()
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local(FAISS_INDEX_FOLDER)
else:
    print("ایندکس قبلی لود شد.")
    vectorstore = FAISS.load_local(FAISS_INDEX_FOLDER, embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# 4. تابع سوال و جواب
def ask(question: str):
    # پیدا کردن قطعات مرتبط
    relevant_docs = retriever.invoke(question)
    context = "\n\n".join([f"منبع: {d.metadata['url']}\n{d.page_content}" for d in relevant_docs])

    # ارسال به OpenRouter
    completion = client.chat.completions.create(
        model="openai/gpt-3.5-turbo",        # یا google/gemini-pro یا anthropic/claude-3-haiku
        messages=[
            {"role": "system", "content": "فقط بر اساس متن زیر جواب بده و در آخر URL منبع را ذکر کن."},
            {"role": "user",   "content": f"متن:\n{context}\n\nسوال: {question}"}
        ],
        temperature=0.3,
        max_tokens=800,
    )

    answer = completion.choices[0].message.content
    sources = [d.metadata['url'] for d in relevant_docs]
    return answer, sources

# ────── تست سریع ──────
if __name__ == "__main__":
    while True:
        q = input("\nسوالت چیه؟ (برای خروج بنویس exit) ")
        if q.lower() in ["exit", "quit", "خروج"]:
            break
        answer, urls = ask(q)
        print("\nپاسخ:")
        print(answer)
        print("\nمنابع:")
        for u in set(urls):
            print("  •", u)